## 기본 합성곱 신경망 구현

In [163]:
import numpy as np
import torch
import torch.nn.functional as F
import warnings
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import os

warnings.filterwarnings("ignore")
os.environ["KMP_DUPLICATE_LIB_OK"]="True"

## 하이퍼 파라미터

In [256]:
epochs = 15

batch_size = 100
no_cuda = False
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

torch.manual_seed(777)
log_interval = 100

## 모델 정의

In [250]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding = 1), # 28 x 28 x 16
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2), # 14 x 14 x 32
            nn.Conv2d(16, 32, 3, padding = 1), # 14 x 14 x 32
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2), # 7 x 7 x 32
            nn.Conv2d(32, 64, 3, padding = 1), # 7 x 7 x 64
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2) # 7 x 7 x 64
        )
        self.fc = nn.Sequential(
            nn.Linear(576, 2048),
            nn.ReLU(),
            nn.Linear(2048, 10)
        )
        

    def forward(self, x):
        out = self.layer(x)
        out = out.view(out.size(0), -1) # 
        out = self.fc(out)
        return F.softmax(out, dim = 1)
        

## 학습, 테스트 루프 정의

## 데이터셋 준비


In [234]:
train_data = datasets.MNIST("../data", train = True, download = True,
                     transform = transforms.Compose([
                         transforms.ToTensor()
                     ]))

train_loader = torch.utils.data.DataLoader(train_data, 
                                          batch_size = batch_size,
                                          shuffle = True)

test_data = datasets.MNIST("../data", train = False, download = True,
                     transform = transforms.Compose([
                         transforms.ToTensor()
                     ]))

test_loader = torch.utils.data.DataLoader(test_data, 
                                          batch_size = batch_size,
                                          shuffle = True)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [235]:
# # Create model
# model = ConvNet()

# # Define loss and optimizer
# loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
# optimizer = tf.keras.optimizers.Adam()

# # Define performance metrics
# train_loss = tf.keras.metrics.Mean(name='train_loss')
# train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

# test_loss = tf.keras.metrics.Mean(name='test_loss')
# test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [236]:
model = ConvNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.1)

from torch.optim.lr_scheduler import ReduceLROnPlateau

# factor는 기존 값 * factor
scheduler = ReduceLROnPlateau(optimizer, mode = "max", factor = 0.1, 
                              patience=0, verbose = True)

## 학습 루프 동작

In [237]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    accuracy = 100 * correct / len(test_loader.dataset)

    # Learning Rate Shecduler
    scheduler.step(accuracy, epoch)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302680
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.361151
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.381151
Train Epoch: 1 [30000/60000 (50%)]	Loss: 2.361151
Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.351151
Train Epoch: 1 [50000/60000 (83%)]	Loss: 2.401151

Test set: Average loss: 2.3638, Accuracy: 974/10000 (10%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.391151
Train Epoch: 2 [10000/60000 (17%)]	Loss: 2.341151
Train Epoch: 2 [20000/60000 (33%)]	Loss: 2.381151
Train Epoch: 2 [30000/60000 (50%)]	Loss: 2.361151
Train Epoch: 2 [40000/60000 (67%)]	Loss: 2.331151
Train Epoch: 2 [50000/60000 (83%)]	Loss: 2.381151
Epoch     2: reducing learning rate of group 0 to 1.0000e-02.

Test set: Average loss: 2.3638, Accuracy: 974/10000 (10%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.301151
Train Epoch: 3 [10000/60000 (17%)]	Loss: 2.321151
Train Epoch: 3 [20000/60000 (33%)]	Loss: 2.391151
Train Epoch: 3 [30000/60000 (50%)]	Loss: 2.341151
Train Epoch: 3 [40000/60000 (67%)